In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab3.ipynb")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy.stats as stats

---

<h1><center>SDSE Lab 3 <br><br> K-means clustering </center></h1>

---

# 1) Load the data
We load the dataset from the file `'data2d.pickle'`. The data matrix is called `X`. Save the number of samples (rows) as `N` and the number of features (columns) as `D`.

In [ ]:
with open('data2d.pickle','rb') as f:
    X = pickle.load(f)
N = X.shape[0]
D = X.shape[1]

In [ ]:
grader.check("q1")

# 2) Scatter plot
Create a scatter plot with the first column of `X` along the x-axis and the second column along the y-axis. Use `plt.axis('equal')` to force the two axes to have the same scale. This is important for correct visualization of the Euclidean distances between points. Your figure should look like the one below.

Food for thought: How many clusters do you see?

<img src="2.png" />

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax.plot(X[:,0], X[:,1], '.')
ax.axis('equal')

# 3) Random placement of centroids
Write a function that takes an integer `K` and the data matrix `X` as inputs, and returns `K` centroids randomly placed within the box spanned by the columns of `X`. The shape of the return value should be `(K,D)`. 

The template also takes a third optional parameter. This is used by the autograder to ensure repeatable results. 

**Note**: You should use `stats.uniform.rvs(loc=..., scale=..., size=...)` to generate random centroid placements. 

In [ ]:
def initialize_centroids(K,X,random_seed=None):
    if random_seed is not None:
        np.random.seed(seed=random_seed)
    D = X.shape[1]
    C = np.empty((K,D))
    xmin = X.min(axis=0)
    xmax = X.max(axis=0)
    for d in range(D):
        C[:,d] = stats.uniform.rvs(loc=xmin[d], scale=xmax[d]-xmin[d], size=K )
    return C

In [ ]:
C4 = initialize_centroids(4,X,random_seed=234)
C8 = initialize_centroids(8,X,random_seed=154)

In [ ]:
grader.check("q3")

# 4) Compute the assignment for given centroids
Write a function that takes the data matrix `X` (shape `(N,D)`) and centroid locations `C` (shape `(K,D)`) as inputs, and returns an assignment matrix `gamma`. `gamma` should be a numpy array with shape `(N,K)`. Each row of `gamma` should contain `K-1` `False` values and one `True` value in the position corresponding to the centroid that is nearest to that data point (Euclidean distance). Hint: `np.argmin`.

In [ ]:
def assign_samples_to_nearest_centroid(X,C):
    N = X.shape[0]
    K = C.shape[0]
    gamma = np.zeros((N,K),dtype=bool)
    for n in range(N):
        kind = np.argmin(np.sum((C-X[n,:])**2,axis=1))
        gamma[n,kind] = True
    return gamma

In [ ]:
gamma4 = assign_samples_to_nearest_centroid(X,C4)
gamma8 = assign_samples_to_nearest_centroid(X,C8)

In [ ]:
grader.check("q4")

# 5) Plot data, centroids, assignment
Write a function that takes as input the data matrix `X`, a centroids matrix `C`, and an assignment matrix `gamma`, and creates a scatter plot similar to the one shown here. The centroids should be indicated with a large dot, and the data samples should be color-coded according to their cluster membership. Your plot may not look identical to this one due to the random placement of the centroids.

<img src="5.png" />


In [ ]:
def plot_scatter(X, C, gamma):
    K = gamma.shape[1]
    fig, ax = plt.subplots(figsize=(5,5))
    for k in range(K):
        ax.plot(X[gamma[:,k],0],X[gamma[:,k],1],'.', markersize=10)
    ax.plot(C[:,0],C[:,1], 'o', markersize=10)
    ax.axis('equal')

In [ ]:
plot_scatter(X, C4, gamma4)

# 6) Compute the centroids for a given assignment
Write a function that takes the assignment `gamma` and the data matrix `X` and returns centroids placed at the mean of each cluster. The return value should be a numpy array with shape `(K,D)`.

In [ ]:
def place_centroids(gamma, X):
    K = gamma.shape[1]
    D = X.shape[1]
    C = np.empty((K,D))
    for k in range(K):
        C[k,:] = np.mean(X[gamma[:,k],:],axis=0)
    return C

In [ ]:
newC4 = place_centroids(gamma4, X)
newC8 = place_centroids(gamma8, X)

In [ ]:
grader.check("q6")

# 7) Code K-means
Now we will implement the K-means algorithm using the methods that we've coded so far. Implement the K-means algorithm described in class. 

The function `run_kmeans` takes the dataset `X` and initial centroids `C` as input. It returns the final placement of the centroids `C`, and the assignment matrix `gamma`. There are two stopping conditions:
+ Stop and return `C=None` and `gamma=None` if any of the clusters becomes empty. 
+ Stop if the assignment matrix does not change from one iteration to the next. 

In [ ]:
def run_kmeans(X,C):

    done = False
    first = True
        
    while not done:
        
        # compute assignment (E step)
        gamma = assign_samples_to_nearest_centroid(X,C)
        
        # place centroids (M step)
        C = place_centroids(gamma, X)
        
        # Break if any of the clusters are empty
        # Return C=None and gamma=None
        if np.any(np.isnan(C)):
            C = None
            gamma = None
            break
            
        # Break if there is no change in the assignment
        if (not first) and np.all(gammaold==gamma):
            break
        
        # update gamma
        gammaold = gamma
        first = False

    return C, gamma

In [ ]:
optC4, optgamma4 = run_kmeans(X,C4)
optC8, optgamma8 = run_kmeans(X,C8)

In [ ]:
grader.check("q7")

# 8) Evaluate the cost function 
Write a function that evaluates the cost function for a given dataset `X`, centroid placement `C`, and assignment matrix `gamma`. The cost function is the sum of the squared distances from each point (row in `X`) to its assigned centroid (row in `C`). The function should return `np.nan` if `C` is `None`.

In [ ]:
def eval_cost(X,C,gamma):
    if C is None:
        return np.nan
    K = gamma.shape[1]
    cost = sum([np.sum((X[gamma[:,k],:] - C[k,:])**2) for k in range(K)])
    return cost

In [ ]:
J4 = eval_cost(X,optC4,optgamma4)
J10 = eval_cost(X,optC8,optgamma8)

In [ ]:
grader.check("q8")

# 9) Ensemble run
Write a function that runs K-means multiple times, each with a different initial centroid placement. These initial placements should be chosen with `initialize_centroids(K,X)`. The function returns `best_run`, which is a dictionary containing the centroids (`C`), cost (`cost`), and assignment matrix (`gamma`) for the best of the ensemble of runs. 

Note that the function also takes `random_seed` as a fourth argument. This is used by the autograder to ensure repeatability of the results. You should **not** pass `random_seed` to `initialize_centroids`.

In [ ]:
def ensemble_run(K,X,ensemblesize,random_seed=None):
    if random_seed is not None:
        np.random.seed(seed=random_seed)
    best_run = {'C':None,'gamma':None,'cost':None}

    for e in range(ensemblesize):
        
        # Run Kmeans with K randomly initialized centroids
        # If an empty cluster is obtained, then repeat, and keep on repeating 
        # until the result has no empty clusters
        done = False
        while not done:
            C = initialize_centroids(K,X)
            C, gamma = run_kmeans(X,C)
            done = C is not None
            
        # Evaluate the cost function 
        cost = eval_cost(X,C,gamma)
        
        # Check whether this is the best result obtained so far, 
        # and if so, store it in best_run.
        if e==0:
            best_run = {'C':C, 'gamma':gamma, 'cost':cost}
        else:
            if cost < best_run['cost']:
                best_run = {'C':C, 'gamma':gamma, 'cost':cost}

    return best_run

In [ ]:
best_run_K4 = ensemble_run(4,X,ensemblesize=100,random_seed=5423)

# sort the result for the autograder to work
ind = np.argsort(np.linalg.norm(best_run_K4['C'],axis=1))
best_run_K4['C'] = best_run_K4['C'][ind,:]
best_run_K4['gamma'] = best_run_K4['gamma'][:,ind]

In [ ]:
grader.check("q9")

# 10) Iteration over K
Complete the function below that takes the data matrix `X`, an ensemble size `ensemblesize`, and a maximum number of clusters to consider (`maxK`).  The function should run `ensemble_run` for all values of `K` from 1 to `maxK`, and return a numpy array of length `maxK` with the best case cost for each value of `K`. 

A random seed is passed in to the function for the autograder. Please pass this seed on to the `ensemble_run` function.

In [ ]:
def K_iteration(X, ensemblesize, maxK,random_seed=None):
    if random_seed is not None:
        np.random.seed(seed=random_seed)
    best_cost = np.empty(maxK)
    for ind in range(maxK):
        K = ind+1
        print(K)
        best_run = ensemble_run(K,X,ensemblesize,random_seed)
        best_cost[ind] = best_run['cost']
    return best_cost

In [ ]:
ensemblesize=100
maxK=10
best_cost = K_iteration(X, ensemblesize, maxK,random_seed=452)

In [ ]:
grader.check("q10")

# 11) Selecting K

Running the cell below will produce a plot showing the performance of the algorithm for different values of `K` ranging from 1 to 10 (calculated in the previous part). The top row shows the intertia cost function, which naturally decreases monotonically with increasing `K`. The bottom plot shows the percentage improvement from one value of `K` to the next. What is the best value of `K` to choose according to these plots? 

In [ ]:
fig, ax = plt.subplots(figsize=(8,5),nrows=2,sharex=True)

ax[0].plot(range(1,maxK+1),best_cost,
         linewidth=3,
         marker='o',
         markersize=8)
ax[0].grid()
ax[0].set_ylabel('Inertia')

ax[1].plot(range(2,maxK+1),100*np.abs(np.diff(best_cost)) / best_cost[1:],
         linewidth=3,
         marker='o',
         markersize=8)
ax[1].grid()
ax[1].set_ylabel('Inertia improvement')
ax[1].set_xlabel('K')

In [ ]:
Kideal = 4

In [ ]:
grader.check("q11")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)